 **Analysis of Independent Presidential Candidata Dr. Iitula's Manifesto for 2019 Election Campaign**


 1. **Imports** https://www.kaggle.com/code/jpandeinge/nlp-analysis-of-pdf-documents/notebook


In [ ]:
#!pip install PyPDF2
#!pip install textract
#!pip install autocorrect
#!pip install vaderSentiment
# pip install wordcloud

In [ ]:
import PyPDF2
import warnings
import textract
from autocorrect import Speller
from nltk.tokenize import word_tokenize
import nltk
import re
import string
from nltk.corpus import stopwords, brown
from nltk.tokenize import word_tokenize, sent_tokenize, RegexpTokenizer
from nltk.stem import WordNetLemmatizer
from autocorrect import spell
import pandas as pd
import plotly.graph_objects as go
from collections import defaultdict
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from textblob import TextBlob
import matplotlib.pyplot as plt
import seaborn as sn
from collections import Counter
warnings.filterwarnings("ignore", category=DeprecationWarning)


In [ ]:
def to_lower(text):
    
    # Specll check the words
    spell = Speller(lang='en')

    texts = spell(text)

    return ' '.join([w.lower() for w in word_tokenize(text)])

In [ ]:
def clean_text(lower_case):
    # split text phrases into words
    words = nltk.word_tokenize(lower_case)

    # Create a list of all the punctuations we wish to remove
    punctuations = ['.', ',', '/', '!', '?', ';',
                    ':', '(', ')', '[', ']', '-', '_', '%']

    # Remove all the special characters
    punctuations = re.sub(r'\W', ' ', str(lower_case))

    # Initialize the stopwords variable, which is a list of words ('and', 'the', 'i', 'yourself', 'is') that do not hold much values as key words
    stop_words = stopwords.words('english')

    # Getting rid of all the words that contain numbers in them
    w_num = re.sub('\w*\d\w*', '', lower_case).strip()

    # remove all single characters
    lower_case = re.sub(r'\s+[a-zA-Z]\s+', ' ', lower_case)

    # Substituting multiple spaces with single space
    lower_case = re.sub(r'\s+', ' ', lower_case, flags=re.I)

    # Removing prefixed 'b'
    lower_case = re.sub(r'^b\s+', '', lower_case)

    # Removing non-english characters
    lower_case = re.sub(r'^b\s+', '', lower_case)

    # Return keywords which are not in stop words
    keywords = [
        word for word in words if not word in stop_words and word in punctuations and word in w_num]

    return keywords


In [ ]:
from wordcloud import WordCloud, STOPWORDS
import imageio
import matplotlib.pyplot as plt
import nltk
def wordImage(words):
    # Image used in which our world cloud output will be
    img1 = imageio.imread(
        r"C:\Users\sclifford\OneDrive - Gallarus Industry Solutions\MscDataAnalytics\MScDataAnalyticsCA2\Data\sentiment\itula.jpeg")
    hcmask1 = img1

    # Get 100 words based on the
    words_except_stop_dist = nltk.FreqDist(w for w in words[:100])
    wordcloud = WordCloud(stopwords=set(STOPWORDS), background_color='black',
                        mask=hcmask1, max_font_size=20).generate(" ".join(words_except_stop_dist))
    plt.imshow(wordcloud, interpolation='bilinear')
    fig = plt.gcf()
    fig.set_size_inches(10, 12)
    plt.axis('off')
    plt.title("Top most common 100 words ", fontsize=20)
    plt.tight_layout(pad=0)
    plt.savefig('Manifesto_top_100.jpeg')


In [ ]:


def sentimentPDF(filePath: str):
    open_filename = open(filePath, 'rb')

    ind_manifesto = PyPDF2.PdfReader(open_filename)
    total_pages = len(ind_manifesto.pages)
    #total_pages
    count = 0
    text = ''

    # Lets loop through, to read each page from the pdf file
    while (count < total_pages):
        # Get the specified number of pages in the document
        mani_page = ind_manifesto.pages[count]
        # Process the next page
        count += 1
        # Extract the text from the page
        text += mani_page.extract_text()
        
    if text != '':
        text = text

    else:
        textract.process(open_filename, method='tesseract',
                        encoding='utf-8', langauge='eng')
            
    lower_case = to_lower(text)
    print(lower_case)  
    # Lemmatize the words
    wordnet_lemmatizer = WordNetLemmatizer()

    lemmatized_word = [wordnet_lemmatizer.lemmatize(
        word) for word in clean_text(lower_case)]

    # lets print out the output from our function above and see how the data looks like
    clean_data = ' '.join(lemmatized_word)
    print(clean_data)   
    df = pd.DataFrame([clean_data])
    df.columns = ['script']
    df.index = ['index']
    return df



In [ ]:
# create plotly gauge chart for sentiment

def sentiment_gauge(blob, file):
    fig = go.Figure(go.Indicator(
        mode="gauge+number",
        value=blob.sentiment.polarity,
        domain={'x': [0, 1], 'y': [0, 1]},
        title={'text': f"Sentiment Polarity of {file}"},
        gauge={'axis': {'range': [-1, 1]},
            'steps': [
            {'range': [-1, -0.5], 'color': "red"},
            {'range': [-0.5, 0.5], 'color': "lightgrey"},
            {'range': [0.5, 1], 'color': "lightgreen"}],
            'threshold': {'line': {'color': "black", 'width': 4}, 'thickness': 0.75, 'value': blob.sentiment.polarity}}))

    fig.show()


In [ ]:
import plotly.graph_objects as go

def sentimentBARChart(sentimentTable: pd.DataFrame):
    # Extract x-axis (categories) and y-axis (scores) data
    categories = sentimentTable['file']
    scores = sentimentTable['sentiment']

    # Define color array based on sentiment scores
    colors = ['green' if score >= 0 else 'red' for score in scores]
 
    # Create bar chart with custom colors
    fig = go.Figure(
        data=[go.Bar(x=categories, y=scores, marker=dict(color=colors))],
        layout=go.Layout(title='Sentiment Analysis', yaxis_title='Sentiment Score',yaxis=dict(range=[-1, 1]))
    )

    # Display the chart
    fig.show()



 **5. Preprocess - Bag of Words model**




In [ ]:
#Preprocess - Bag of Words model
#  Counting the occurrences of tokens and building a sparse matrix of documents x tokens.
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np

def preProcessBagOfWords(df: pd.DataFrame):
    corpus = df.script
    vect = CountVectorizer(stop_words='english')

    # Transforms the data into a bag of words
    data_vect = vect.fit_transform(corpus)
    feature_names = vect.get_feature_names_out()
    data_vect_feat = pd.DataFrame(data_vect.toarray(), columns=feature_names)
    data_vect_feat.index = df.index
    data = data_vect_feat.transpose()
    # Find the top 1000 words written in the manifesto
    top_dict = {}
    for c in data.columns:
        top = data[c].sort_values(ascending=False)
        top_dict[c] = list(zip(top.index, top.values))
    #for x in list(top_dict)[0:100]:
        #print("key {}, value {} ".format(x,  top_dict[x]))
    words = []
    for president in data:
        top = [word for (word, count) in top_dict[president]]
        for t in top:
            words.append(t)

    print(words[:10])
    blob = TextBlob(df.script[0])
    blob.sentiment  
    return blob


In [41]:
def sentimentPDFdata(files: list):
    # ccreate a df with 2 columns called file and sentiment
    df_list = pd.DataFrame(columns=['file', 'sentiment'])
    for file in files:
        fileName = file.split('/')[-1]
        blob_ = preProcessBagOfWords(sentimentPDF(file))
        sentiment_gauge(blob_, fileName)
        #add a row to df_list
        
        df_list = pd.concat([df_list, pd.DataFrame({'file': [fileName], 'sentiment': [blob_.sentiment.polarity]})], ignore_index=True)
    return df_list

In [ ]:
filesIreland = ['Data/sentiment/homeBuilding/246610_5161c6c9-559c-4b56-93ad-fe894e7ee4e0.pdf','Data/sentiment/homeBuilding/Construction Sector Performance and Capacity.pdf', 'Data/sentiment/homeBuilding/PII Housing Review September 2022.pdf','Data/sentiment/homeBuilding/SCSI_ResidentialPropertyReport2022_Final.pdf']
#filesIreland = ['Data/sentiment/homeBuilding/PII Housing Review September 2022.pdf']
sentimentIreland_df = sentimentPDFdata(filesIreland)
sentimentBARChart(sentimentIreland_df)
sentimentIreland_df


In [ ]:
filesEurope = ['Data/sentiment/homeBuilding/europe/Emerging Trends in Real Estate Europe 2023 Report.pdf','Data/sentiment/homeBuilding/europe/at-property-index-2022-final.pdf','Data/sentiment/homeBuilding/europe/dp171_en.pdf']
sentimentEurope_df = sentimentPDFdata(filesEurope)
sentimentBARChart(sentimentEurope_df)
sentimentEurope_df

In [42]:
buildCostfilesIreland = ['Data/sentiment/buildingCost/Ireland/2022-05-24_opening-statement-kevin-james-vice-president-society-of-chartered-surveyors-ireland_en.pdf','Data/sentiment/buildingCost/Ireland/256082_afbe94c3-ebf1-4201-9a4a-a6ac9cddc69a.pdf', 'Data/sentiment/buildingCost/Ireland/Rising-construction-costs-and-the-residential-real-estate-market-in-Ireland.pdf']
buildCostsentimentIreland_df = sentimentPDFdata(buildCostfilesIreland)
sentimentBARChart(buildCostsentimentIreland_df)
buildCostsentimentIreland_df

page | 1 society of chartered surveyors ireland opening statement to joint oireachtas committee on housing , local government & heritage on construction costs in housing ( land , design , labour & materials ) introduction the society of chartered surveyors ireland ( sc si ) welcomes the opportunity to share our insights on construction costs to the joint committee on housing , local government and heritage on construction costs and we thank the chairman and committee for the invitation . as the leading professional body for chartered surveying professionals working in the property , construction and land sectors across ireland , we undertake research on a wide range of economic , industry and practice -related issues in the public interest . chartered surveyor members of the scsi work across the built environment , in both the public and private sectors , providing advice to clients across the entire lifecycle of a build , from green field site through to design , mapping , budgets , p

residential construction cost study report may 2023 a shared government and construction sector group i nitiative www.gov.ie i ii table of contents minister ’ s foreword .................................................................................................... 1 glossary of abbreviations .......................................................................................... 1 glossary of terminology ............................................................................................ 5 1.0 executive summary ........................................................................................... 9 2.0 introduction ...................................................................................................... 23 3.0 review of existing literature .............................................................................. 31 4.0 methodology ....................................................................................................... 39 5.0 case st

rising construction costs and the residential real estate market in ireland filippo arigoni , gerard kennedy & neill killeen vol . 2022 , no . 12 rising construction costs and the residential real estate market in ireland filippo arigoni , gerard kennedy & neill killeen1 central bank of ireland october 2022 abstract construction costs are a key factor to consider when analysing the residential real estate market in ireland given their impact on housing supply . this note examines longer -term trends in construction cos ts in ireland and sho ws that these costs have increased steadily over the last twenty -five years and faster than general inflation over the same period . the increase in gross construction costs over this time was driven primarily by cost inflation in two periods , namely the early 2000s and the increases observed since march 2020 . some of these patterns have been accentuated by changes to the tax regime over the period 1998 -2008 . in the last two years , constructio

,file,sentiment
0,2022-05-24_opening-statement-kevin-james-vice-...,0.073203
1,256082_afbe94c3-ebf1-4201-9a4a-a6ac9cddc69a.pdf,0.051398
2,Rising-construction-costs-and-the-residential-...,0.043804


In [43]:
buildCostfilesEurope = ['Data/sentiment/buildingCost/Europe/22-cs11-_Construction_Building_Materials_-_Commentary_October_2022.pdf','Data/sentiment/buildingCost/Europe/ING-Think-eu-construction-outlook-optimism-among-contractors-despite-increasing-building-material-shortage.pdf', 'Data/sentiment/buildingCost/Europe/International Construction Costs 2022-2.pdf']
buildCostsentimentEurope_df = sentimentPDFdata(buildCostfilesEurope)
sentimentBARChart(buildCostsentimentEurope_df)
buildCostsentimentEurope_df

responsible statistician : pio francesco medolla email : materialstats @ beis.gov.uk media enquiries : 020 7215 1000 public enquiries : +44 ( 0 ) 207215 2820 next publication : 7 december 2022 monthly statistics of building materials and components commentary , october 2022 coverage : uk and great britain geographical area : country , region and county 2 november 2022 national statistics h eadline findings •the material price index for ‘ all work ’ increased by 16.7 % in september 2022 compared to the same month the previous year . •there was a 2.6 % increase in brick deliver ies in september 2022 c ompared to september 2021 , according to the seasonally adjus ted figures . •there was a 9.7 % decrease in concrete block deliveries in september 2022 compared to september 2021 , according to the seasonally adjusted figures . c hart 1 : construction material price indices , uk index , 2015 = 100 monthly statistics of building materials and components – october 2022 2 contents introduction 

eu construction outlook : contractors ’ optimism rising despite building material shortages we expect further growth in the eu construction sector this year and next , despite a continuing shortage of some building materials . the eu construction confidence indicator was positive at the start of 2022 . building volumes should be boosted by fresh investment from the eu recovery fund new homes being constructed in stellendam in the netherlands content further increasing material shortages- higher sales ' prices- eu construction confidence indicator positive- issuance of residential building permits stable- europe at a glance- low growth infrastructure sector- huge differences in the long run- more growth expected for eu construction sector- further increasing material shortages the outlook for construction in the european union has changed . back in september last year , we noted in our eu construction outlook that supply chain economic and financial analysis 9 february 2022 articledisru

1 international construction costs 2022 international construction costs 2022the year of inflation2 international construction costs 2022 construction , however , has again proved itself extremely adaptable during the last year in responding to the difficult circumstances . we have seen sustained delivery of housing and infrastructure across most global markets , better use of data , and increasing investment in technological solutions such as modern methods of construction , all of which can improve efficiencies and aid the drive to net-zero . this adaptability will prove vital as businesses ready themselves for the uncertain and inflationary environment ahead . this theme is central to our 2022 international construction costs index , which highlights dramatic price fluctuations in many regions around the globe . while there is healthy confidence in the future of the construction sector internationally , differing government covid-19 strategies have resulted in varying paces of recov

,file,sentiment
0,22-cs11-_Construction_Building_Materials_-_Com...,0.067373
1,ING-Think-eu-construction-outlook-optimism-amo...,0.062333
2,International Construction Costs 2022-2.pdf,0.096486
